# Getting started
The Expected Parrot Domain-Specifc Language (EDSL) library is a collection of tools for conducting surveys and experiments with AI agents. This page provides a series of examples for the basic EDSL functions:

- We create questions using the `Question` class and subclasses for different question types (multiple choice, free text, etc.).
- We combine questions into surveys using the `Survey` class and specify survey logic and question parameters (`Scenario` objects).
- We design AI agents with personas representing survey respondents using the `Agent` class.
- We select language models to use in simulating survey responses from agents using the `Model` class.
- We administer surveys and explore the results using `Results` class methods for analysis and visualization.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/tutorial_getting_started.ipynb)

In [1]:
# ! pip install edsl

## Contents

- [Creating a survey](#creating-a-survey)
- [Question types](#question-types)
- [Survey logic](#survey-logic)
- [Scenarios](#scenarios)
- [Agents](#agents)
- [Models](#models)
- [Exploring results](#exploring-results)

## Creating a survey
We start by creating some survey questions. The EDSL library provides a variety of question types with different formats as subclassess of the `Question` class. 

- All questions require a unique question name and a question text. 
- Question types other than free text and yes/no also require a list of answer options. 

Select a question type based on the form of the response that you want to receive. For example, `QuestionFreeText` will return an unstructured textual response and `QuestionMultipleChoice` will return a single selection from a specified list of options. This is done by prompting the agent to respond to the question in the required format. We cover some details about prompts in the section on [Exploring results](#exploring-results).

### Question types
Here we import each question type and show an example question in the required format:

#### Multiple choice
A multiple choice question prompts the agent to select one of the answer options.

In [2]:
from edsl.questions import QuestionMultipleChoice

q_mc = QuestionMultipleChoice(
    question_name = "q_mc",
    question_text = "How often do you shop for clothes?",
    question_options = [
        "Rarely or never",
        "Annually",
        "Seasonally",
        "Monthly",
        "Daily"
    ]
)

#### Checkbox
A checkbox question requires the agent to select one or more of the answer options, which are returned as a list. The minimum and maximum number of options that may be selected are optional.

In [3]:
from edsl.questions import QuestionCheckBox

q_cb = QuestionCheckBox(
    question_name = "q_cb",
    question_text = """Which of the following factors are important to you in making decisions 
    about clothes shopping? Select all that apply.""",
    question_options = [
        "Price",
        "Quality",
        "Brand Reputation",
        "Style and Design",
        "Fit and Comfort",
        "Customer Reviews and Recommendations",
        "Ethical and Sustainable Practices",
        "Return Policy",
        "Convenience",
        "Other"
    ],
    min_selections = 1, # optional
    max_selections = 3  # optional
)

#### Free text
A free text question prompts the agent to respond with unstructured text.

In [4]:
from edsl.questions import QuestionFreeText

q_ft = QuestionFreeText(
    question_name = "q_ft",
    question_text = "What improvements would you like to see in options for clothes shopping?"
)

#### Linear scale
A linear scale question prompts the agent to select one of the answer options which are a list of integers.

In [5]:
from edsl.questions import QuestionLinearScale

q_ls = QuestionLinearScale(
    question_name = "q_ls",
    question_text = "On a scale of 0-10, how much do you typically enjoy clothes shopping?",
    question_options = [0,1,2,3,4,5,6,7,8,9,10],
    option_labels = {0:"Not at all", 10:"Very much"}
)

#### Numerical
A numerical question prompts the agent to respond with a number.

In [6]:
from edsl.questions import QuestionNumerical

q_nu = QuestionNumerical(
    question_name = "q_nu",
    question_text = """Estimate the amount of money that you spent shopping for clothes in 
    the past year (in $USD)."""
)

#### List
A list question prompts the agent to provide a response in the form of a list.

In [7]:
from edsl.questions import QuestionList

q_li = QuestionList(
    question_name = "q_li",
    question_text = "What improvements would you like to see in options for clothes shopping?"
)

#### Budget
A budget question prompts the agent to allocation a sum among the answer options and respond with a dictionary where the keys are the answer options and the values are the allocated amounts.

In [8]:
from edsl.questions import QuestionBudget

q_bg = QuestionBudget(
    question_name = "q_bg",
    question_text = """Estimate the percentage of your total time spent shopping for clothes 
    in each of the following modes.""",
    question_options=[
        "Online",
        "Malls",
        "Freestanding stores",
        "Mail order catalogs",
        "Other"
    ],
    budget_sum = 100,
)

#### Yes / No
A yes/no question prompts the agent to respond with a yes or no. The answer options are pre-set. Use a multiple choice or other appropriate type if you want to include other options (e.g., "I don't know").

In [9]:
from edsl.questions import QuestionYesNo

q_yn = QuestionYesNo(
    question_name = "q_yn",
    question_text = "Have you ever felt excluded or frustrated by options for clothes shopping?", 
)

### Survey logic
We combine questions into a survey using the `Survey` class. A `Survey` object is created with a list of one or more questions:

In [10]:
from edsl import Survey 

clothes_shopping_survey = Survey(questions = [q_mc, q_cb, q_ft, q_ls, q_nu, q_li, q_bg, q_yn])

By default, questions are delivered asynchronously to agents when we run a survey. We can specify different survey logic with the `add_stop_rule` and `add_targeted_memory` methods.

#### Stop/skip logic
Here we create a survey with 2 questions and apply a rule to skip the second question based on a negative response to the first question. We can use any rule that can be expressed as a logical statement. The form of the method is `Survey.add_stop_rule(question_name, <logical expression>)`:

In [11]:
q1 = QuestionYesNo(
    question_name = "enjoy",
    question_text = "Do you enjoy clothes shopping?"
)
q2 = QuestionLinearScale(
    question_name = "enjoy_scale",
    question_text = """On a scale of 0-10, how much do you typically enjoy clothes shopping?""",
    question_options = [0,1,2,3,4,5,6,7,8,9,10],
    option_labels = {0:"Not at all", 10:"Very much"}
)
survey = Survey(questions = [q1, q2])
survey.add_stop_rule("enjoy", "enjoy == 'No'")

Survey(questions=[QuestionYesNo(question_name = 'enjoy', question_text = 'Do you enjoy clothes shopping?', question_options = ['Yes', 'No'], short_names_dict = {}), QuestionLinearScale(question_name = 'enjoy_scale', question_text = 'On a scale of 0-10, how much do you typically enjoy clothes shopping?', question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], short_names_dict = {}, option_labels = {0: 'Not at all', 10: 'Very much'})], name=None)

#### Question memory
The `add_targeted_memory` method lets us change the default asynchronous behavior of the survey and selectively include questions and responses in subsequent question prompts as agent memories. Here we add the question and response for the first question to the prompt for the second question:

In [12]:
from edsl.questions import QuestionYesNo, QuestionLinearScale
from edsl import Survey

q1 = QuestionYesNo(
    question_name = "enjoy",
    question_text = "Do you enjoy clothes shopping?"
)
q2 = QuestionLinearScale(
    question_name = "enjoy_scale",
    question_text = "On a scale of 0-10, how much do you typically enjoy clothes shopping?",
    question_options = [0,1,2,3,4,5,6,7,8,9,10],
    option_labels = {0:"Not at all", 10:"Very much"}
)
survey = Survey(questions = [q1, q2])
survey.add_targeted_memory(q2, q1)

Survey(questions=[QuestionYesNo(question_name = 'enjoy', question_text = 'Do you enjoy clothes shopping?', question_options = ['Yes', 'No'], short_names_dict = {}), QuestionLinearScale(question_name = 'enjoy_scale', question_text = 'On a scale of 0-10, how much do you typically enjoy clothes shopping?', question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], short_names_dict = {}, option_labels = {0: 'Not at all', 10: 'Very much'})], name=None)

Note that we can specify the question names or ids as the arguments to the `add_targeted_memory` method. The following command is equivalent:

In [13]:
# survey.add_targeted_memory("enjoy_scale", "enjoy")

We can do this as many times as desired to add multiple prior questions and answers to the prompt for a subsequent question (we show how to examine the prompts below in [Prompts](#prompts)). Here we add 2 prior questions and responses to a question:

In [14]:
q3 = QuestionFreeText(
    question_name = "pros_cons",
    question_text = "What are some pros and cons of online clothes shopping?"
)
survey = Survey([q1, q2, q3])
survey.add_targeted_memory(q3, q1)
survey.add_targeted_memory(q3, q2)

Survey(questions=[QuestionYesNo(question_name = 'enjoy', question_text = 'Do you enjoy clothes shopping?', question_options = ['Yes', 'No'], short_names_dict = {}), QuestionLinearScale(question_name = 'enjoy_scale', question_text = 'On a scale of 0-10, how much do you typically enjoy clothes shopping?', question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], short_names_dict = {}, option_labels = {0: 'Not at all', 10: 'Very much'}), QuestionFreeText(question_name = 'pros_cons', question_text = 'What are some pros and cons of online clothes shopping?', allow_nonresponse = False)], name=None)

### Scenarios
We can parameterize our questions in order to create different versions of "scenarios" of them using the `Scenario` class. Here we create a question with a parameter and then create scenarios for the parameter values that we want to use in the survey:

In [15]:
from edsl import Scenario

q = QuestionFreeText(
    question_name = "favorite",
    question_text = "Describe your favorite {{ item }}."
)

scenarios = [Scenario({"item": i}) for i in ["t-shirt", "hat", "sweater"]]

Scenarios are applied to a survey using the `by` method. This allows the scenarios to be applied to all questions in the survey that take the same parameters. We typically wait to do this until we are administering the survey using the `run` method:

In [16]:
results = q.by(scenarios).run()

Note that we can administer a single question individually because the `run` method automatically converts it into a `Survey` object for convenience. The following commands will yield an identical result to the command above:

In [17]:
survey = Survey([q])
results = survey.by(scenarios).run()

### Agents
We can create personas and traits for the AI agents that will respond to our survey with the `Agent` class. An `Agent` object takes an optional `name` and a (required) set of `traits` in the form of a dictionary. We can use the `example` method to see an example:

In [18]:
from edsl import Agent 

Agent.example()

Agent(traits = {'age': 22, 'hair': 'brown', 'height': 5.5})

Here we create a single agent with a name and a short persona description:

In [19]:
agent = Agent(
    name = "Agent_shopper", 
    traits = {"persona": "You are a 45-year-old woman who prefers to shop online."}
    )
agent

Agent(name = 'Agent_shopper', traits = {'persona': 'You are a 45-year-old woman who prefers to shop online.'})

Here we use lists of traits to create a panel of agents with combinations of the dimensions. For convenience in analyzing results by agent traits later on (e.g., by agent age), we can include the trait dimensions both individually and as part of the persona narrative:

In [20]:
base_persona = "You are a {age}-year-old {height} woman who prefers to shop online."
ages = [25, 45, 65,]
heights = ["short", "average", "tall"]

agents = [Agent(traits = {
    "persona": base_persona.format(age = age, height = height),
    "age": age,
    "height": height
    }) for age in ages for height in heights]
agents

[Agent(traits = {'persona': 'You are a 25-year-old short woman who prefers to shop online.', 'age': 25, 'height': 'short'}),
 Agent(traits = {'persona': 'You are a 25-year-old average woman who prefers to shop online.', 'age': 25, 'height': 'average'}),
 Agent(traits = {'persona': 'You are a 25-year-old tall woman who prefers to shop online.', 'age': 25, 'height': 'tall'}),
 Agent(traits = {'persona': 'You are a 45-year-old short woman who prefers to shop online.', 'age': 45, 'height': 'short'}),
 Agent(traits = {'persona': 'You are a 45-year-old average woman who prefers to shop online.', 'age': 45, 'height': 'average'}),
 Agent(traits = {'persona': 'You are a 45-year-old tall woman who prefers to shop online.', 'age': 45, 'height': 'tall'}),
 Agent(traits = {'persona': 'You are a 65-year-old short woman who prefers to shop online.', 'age': 65, 'height': 'short'}),
 Agent(traits = {'persona': 'You are a 65-year-old average woman who prefers to shop online.', 'age': 65, 'height': 'aver

Agents are assigned to a survey using the `by` method. Similar to scenarios, this will administer all questions to each agent. We also typically wait to do this until we are administering the survey using the `run` method:

In [21]:
results = survey.by(scenarios).by(agents).run()

It does not matter which order we use the `by` method to add the scenarios and agents. The following command will yield an identical result as the command above:

In [22]:
results = survey.by(agents).by(scenarios).run()

However, if agents or scenarios are created independently they must be combined in the same `by` method call as a list. For example, we could create agents separately and then combine them in a list:

In [23]:
agent_1 = Agent(traits = {"persona": "You are a 45-year-old woman who prefers to shop online."})
agent_2 = Agent(traits = {"persona": "You are a 25-year-old man who prefers to shop in person."})

results = survey.by(scenarios).by([agent_1, agent_2]).run()

Note that if an agent name is not specified when the `Agent` object is created, an `agent_name` field is automatically created when survey results are generated. We will see this when we inspect results in the next sections.

### Models
We use the `Model` class to specify the language models that we want to use in simulating survey results. The default model is GPT-4 (if we do not specify a model - as in the above examples - the results are generated using the default model). We can use the `available` method to see a list of available models:

In [24]:
from edsl import Model

Model.available()

['claude-3-haiku-20240307',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'dbrx-instruct',
 'gemini_pro',
 'gpt-3.5-turbo',
 'gpt-4-1106-preview',
 'llama-2-13b-chat-hf',
 'llama-2-70b-chat-hf',
 'mixtral-8x7B-instruct-v0.1']

Here we create a list of models to call in running the survey - results will be generated using each of them:

In [25]:
models = [Model(m) for m in ["gpt-3.5-turbo", "gpt-4-1106-preview"]]

We can call the objects to see all of the parameters that we might choose to adjust (e.g., temperature):

In [26]:
models

[LanguageModelOpenAIThreeFiveTurbo(model = 'gpt-3.5-turbo', parameters={'temperature': 0.5, 'max_tokens': 1000, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'logprobs': False, 'top_logprobs': 3}),
 LanguageModelOpenAIFour(model = 'gpt-4-1106-preview', parameters={'temperature': 0.5, 'max_tokens': 1000, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'logprobs': False, 'top_logprobs': 3})]

Models are also assigned to a survey using the `by` method. We typically wait to do this until we are administering the survey using the `run` method:

In [27]:
results = survey.by(scenarios).by(agents).by(models).run()

## Exporing results
We can reference the components of the results with the `columns` method. This command will return a list of all the fields in the results that may be selected individually:

In [28]:
results.columns

['agent.age',
 'agent.agent_name',
 'agent.height',
 'agent.persona',
 'answer.favorite',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.favorite_system_prompt',
 'prompt.favorite_user_prompt',
 'raw_model_response.favorite_raw_model_response',
 'scenario.item']

We can do the same for our clothes shopping survey and see the question- and agent-specific components of the results:

In [29]:
results_shopping = clothes_shopping_survey.by(agents).by(models).run()
results_shopping.columns

['agent.age',
 'agent.agent_name',
 'agent.height',
 'agent.persona',
 'answer.q_bg',
 'answer.q_bg_comment',
 'answer.q_cb',
 'answer.q_cb_comment',
 'answer.q_ft',
 'answer.q_li',
 'answer.q_li_comment',
 'answer.q_ls',
 'answer.q_ls_comment',
 'answer.q_mc',
 'answer.q_mc_comment',
 'answer.q_nu',
 'answer.q_nu_comment',
 'answer.q_yn',
 'answer.q_yn_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.q_bg_system_prompt',
 'prompt.q_bg_user_prompt',
 'prompt.q_cb_system_prompt',
 'prompt.q_cb_user_prompt',
 'prompt.q_ft_system_prompt',
 'prompt.q_ft_user_prompt',
 'prompt.q_li_system_prompt',
 'prompt.q_li_user_prompt',
 'prompt.q_ls_system_prompt',
 'prompt.q_ls_user_prompt',
 'prompt.q_mc_system_prompt',
 'prompt.q_mc_user_prompt',
 'prompt.q_nu_system_prompt',
 'prompt.q_nu_user_prompt',
 'prompt.q_yn_system_prompt',
 '

Note that each question type other than free text also automatically includes a `_comment` field where the agent is allowed to provide unstructed textual commentary in addition to its required formatted response. We'll show how to access these fields in the next sections.

### Printing
We can use some print methods to display results in readable formats. The basic `print` method displays components horizontally, which can be unwieldy when we do this for all components at once, even for a survey consisting of a single question (we show how to narrow the set of components displayed in next steps):

In [30]:
results[0].print()

                                                      Result                                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Attribute              ┃ Value                                                                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ agent                  │                                    Agent Attributes                                    │
│                        │ ┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│                        │ ┃ Attribute               ┃ Value                                                    ┃ │
│                        │ ┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│                        │ │ _name                   │ None                                                     │ │
│                        │ │ _traits                 │ {'persona': 'You are a 25-year-old short woman who       │ │
│                        │ │                         │ prefers to shop online.', 'age': 25, 'height': 'short'}  │ │
│                        │ │ _codebook               │ {}                                                       │ │
│                        │ │ _instruction            │ 'You are answering questions as if you were a human. Do  │ │
│                        │ │                         │ not break character.'                                    │ │
│                        │ │ set_instructions        │ False                                                    │ │
│                        │ │ dynamic_traits_function │ None                                                     │ │
│                        │ │ current_question        │ QuestionYesNo(question_name = 'q_yn', question_text =    │ │
│                        │ │                         │ 'Have you ever felt excluded or frustrated by options    │ │
│                        │ │                         │ for clothes shopping?', question_options = ['Yes',       │ │
│                        │ │                         │ 'No'], short_names_dict = {}, model_instructions = {})   │ │
│                        │ └─────────────────────────┴──────────────────────────────────────────────────────────┘ │
│ scenario               │         Scenario Attributes                                                            │
│                        │ ┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓                                                    │
│                        │ ┃ Attribute ┃ Value               ┃                                                    │
│                        │ ┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩                                                    │
│                        │ │ data      │ {'item': 't-shirt'} │                                                    │
│                        │ └───────────┴─────────────────────┘                                                    │
│ model                  │                                     Language Model                                     │
│                        │ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│                        │ ┃ Attribute                   ┃ Value                                                ┃ │
│                        │ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│                        │ │ model                       │ 'gpt-3.5-turbo'                                      │ │
│                        │ │ parameters                  │ {'temperature': 0.5, 'max_tokens': 1000, 'top_p': 1, │ │
│                        │ │                             │ 'frequency_penalty': 0, 'presence_penalty': 0,       │ │
│                        │ │                            

### Select
We can use the `select` method together with the `print` method to narrow the components of results that we display. Here we select some of the agent parameters and the response to the (single) question in the survey. Note that we can drop the prefixes `agent.`, `answer.`, etc., in the select statement or include them for clarity. The following commands will print the same results:

In [31]:
# results.select("agent.age", "agent.height", "scenario.item", "answer.favorite").print()
results.select("age", "height", "item", "favorite").print()

agent.age,agent.height,scenario.item,answer.favorite
25,short,t-shirt,My favorite t-shirt is a cozy oversized black shirt with a simple white graphic design on the front. It's super comfortable and goes well with any pair of jeans or leggings.
25,short,t-shirt,"My favorite t-shirt is a soft, vintage-style band tee that I stumbled upon online. It's a heather gray with a classic rock band's iconic logo emblazoned across the chest. The material is a cotton blend, making it super comfortable and perfect for everyday wear. It's slightly oversized, which I love because it gives off a relaxed, effortless vibe. It's been through countless washes, but it still maintains its shape and the print hasn't faded. It's my go-to piece when I want to combine comfort with a touch of edgy style."
25,short,hat,My favorite hat is a cozy beanie with a cute pom-pom on top. It keeps me warm during the winter months and adds a fun touch to my outfits.
25,short,hat,"My favorite hat is a cozy, knit beanie with a faux fur pom-pom on top. It's a soft shade of blush pink that goes well with almost everything in my wardrobe. It's snug enough to stay put without being too tight, and it keeps my head warm during the chilly months. I found it on a small boutique website last winter, and it's been my go-to ever since."
25,short,sweater,My favorite sweater is a cozy oversized knit in a soft pink color. It's perfect for lounging around the house or pairing with jeans for a casual day out.
25,short,sweater,"My favorite sweater is a cozy, oversized knit that I found online. It's a lovely shade of dusty rose, which complements my skin tone perfectly. The sweater has a chunky turtleneck that keeps me warm without feeling too constrictive, and the long sleeves can be pulled down over my hands, which I love to do when I'm feeling particularly chilly. It's long enough to pair with leggings for a comfortable, yet put-together look. The best part is the softness of the material—it feels like a warm hug every time I put it on."
25,average,t-shirt,"My favorite t-shirt is a soft, oversized white shirt with a cute graphic print on the front. It's super comfortable and goes well with jeans or leggings."
25,average,t-shirt,"My favorite t-shirt is a soft, heather gray number with a relaxed fit. It has a scoop neckline and short sleeves that are just the right length—not too short, not too long. The front features a minimalistic design, a small embroidered heart near the hem that adds a subtle touch of personality without being too loud. It's made from a cotton blend which makes it super comfortable and easy to care for. I love that it's versatile enough to pair with jeans for a casual look or with a blazer for a more put-together outfit."
25,average,hat,My favorite hat is a cozy knit beanie in a light gray color. It's perfect for keeping me warm during the colder months and goes well with most of my outfits.
25,average,hat,"My favorite hat is a cozy, chunky knit beanie that I found online last winter. It's a beautiful shade of dusty rose, which goes well with most of my wardrobe. It's super warm and has a ribbed texture, with a cute little faux fur pom-pom on top. I love how it keeps me warm while adding a stylish touch to my outfits on chilly days."


### Filtering results
We can filter our responses by adding a logical expression to the `filter` method. Here we filter results to those provided by GPT-4 where the response to question `q_nu` is greater than 1000, and then examine the response and `_comment` to question `q_li` together with the response to `q_nu`. We include the `agent_name` to show the names that were generated for the agents as we did not specify them in creating the agents:

In [32]:
(results_shopping
.filter("q_nu > str(1000) and model.model == 'gpt-4-1106-preview'")
.select("agent_name", "answer.q_nu", "answer.q_li", "answer.q_li_comment")
.print()
)

agent.agent_name,answer.q_nu,answer.q_li,answer.q_li_comment
Agent_0,500,"['virtual fitting rooms', 'comprehensive size filters', 'user reviews with photos', 'detailed size guides', '360-degree product views', 'better return policies', 'augmented reality features', 'style recommendations', 'inclusive sizing']","As someone who shops online frequently, I'd love to see these features improved to make the experience more convenient and tailored to individual needs, especially for those of us with unique body types."
Agent_3,500,"['more size inclusivity', 'better size filters', '360-degree product views', 'virtual fitting rooms', 'detailed fabric information', 'user reviews with photos', 'easy return policies', 'sustainability information']","As someone who prefers to shop online, I find these improvements would greatly enhance the shopping experience, ensuring a better fit and more informed purchases."
Agent_5,500,"['virtual fitting rooms', 'detailed size guides', 'user-friendly mobile interface', '360-degree product views', 'advanced search filters', 'personalized recommendations', 'easy return process', 'sustainable options', 'real-time customer service', 'inclusive sizing']","As someone who prefers to shop online, these improvements would greatly enhance the shopping experience, ensuring a better fit, easier navigation, and more informed purchases."
Agent_4,500,"['more size inclusivity', 'better size filters', 'virtual try-on features', 'user reviews with photos', 'sustainable clothing options', 'detailed fabric information', 'easy return policies']","As someone who shops online frequently, I find that these improvements would greatly enhance the shopping experience, especially when it comes to ensuring a good fit and making informed choices."
Agent_1,500,"['virtual fitting rooms', 'better size standardization', 'advanced search filters', '360-degree product views', 'customer reviews with photos', 'sustainable fashion options', 'easy return processes', 'detailed product descriptions', 'inclusive sizing', 'real-time inventory updates']","These improvements would greatly enhance the online shopping experience, making it easier to find the right products and reduce the number of returns due to sizing or quality issues."
Agent_6,500,"['petite sizes', '360-degree views', 'virtual try-on', 'easy returns', 'detailed size charts', 'customer reviews', 'filter by height', 'adaptive clothing']","As a short woman, I often struggle to find clothes that fit well without having to alter them. I'd love to see more options for petite sizes and better visual tools like 360-degree views and virtual try-on features to get a sense of how clothes might fit. Easy returns are crucial since things don't always work out. Detailed size charts and customer reviews help make more informed choices. Filters by height would be so helpful, and more options for adaptive clothing would be appreciated for those with different accessibility needs."
Agent_8,500,"['better size filters', '360-degree product views', 'virtual try-on', 'detailed fabric information', 'user-friendly return policies', 'customer reviews with photos', 'petite and tall categories', 'adaptive clothing options']","As a tall woman, I find it challenging to get the right fit online. Improved size filters and categories for different body types would be beneficial. Virtual try-on technology and 360-degree views could help visualize how clothes fit. Detailed fabric information is crucial for comfort and durability decisions. User-friendly return policies would make the process less risky, and customer reviews with photos help gauge real-life appearances. Adaptive clothing options would also be a thoughtful inclusion for those with different physical needs."
Agent_2,1200,"['virtual fitting rooms', 'comprehensive size filters', 'user reviews with photos', '360-degree product views', 'augmented reality features', 'personalized recommendations', 'easy return policies', 'sustainable fashion options']","

### Table labels
We can also apply some `pretty_labels` to our tables in the `print` method. Note that we do need to include the object prefixes in specifying the labels in the print statement:

In [33]:
(results
.select("age", "height","item", "favorite")
.print(pretty_labels = {
    "agent.age": "Age",
    "agent.height": "Height",
    "scenario.item": "Item",
    "answer.favorite": q.question_text
})
)

Age,Height,Item,Describe your favorite {{ item }}.
25,short,t-shirt,My favorite t-shirt is a cozy oversized black shirt with a simple white graphic design on the front. It's super comfortable and goes well with any pair of jeans or leggings.
25,short,t-shirt,"My favorite t-shirt is a soft, vintage-style band tee that I stumbled upon online. It's a heather gray with a classic rock band's iconic logo emblazoned across the chest. The material is a cotton blend, making it super comfortable and perfect for everyday wear. It's slightly oversized, which I love because it gives off a relaxed, effortless vibe. It's been through countless washes, but it still maintains its shape and the print hasn't faded. It's my go-to piece when I want to combine comfort with a touch of edgy style."
25,short,hat,My favorite hat is a cozy beanie with a cute pom-pom on top. It keeps me warm during the winter months and adds a fun touch to my outfits.
25,short,hat,"My favorite hat is a cozy, knit beanie with a faux fur pom-pom on top. It's a soft shade of blush pink that goes well with almost everything in my wardrobe. It's snug enough to stay put without being too tight, and it keeps my head warm during the chilly months. I found it on a small boutique website last winter, and it's been my go-to ever since."
25,short,sweater,My favorite sweater is a cozy oversized knit in a soft pink color. It's perfect for lounging around the house or pairing with jeans for a casual day out.
25,short,sweater,"My favorite sweater is a cozy, oversized knit that I found online. It's a lovely shade of dusty rose, which complements my skin tone perfectly. The sweater has a chunky turtleneck that keeps me warm without feeling too constrictive, and the long sleeves can be pulled down over my hands, which I love to do when I'm feeling particularly chilly. It's long enough to pair with leggings for a comfortable, yet put-together look. The best part is the softness of the material—it feels like a warm hug every time I put it on."
25,average,t-shirt,"My favorite t-shirt is a soft, oversized white shirt with a cute graphic print on the front. It's super comfortable and goes well with jeans or leggings."
25,average,t-shirt,"My favorite t-shirt is a soft, heather gray number with a relaxed fit. It has a scoop neckline and short sleeves that are just the right length—not too short, not too long. The front features a minimalistic design, a small embroidered heart near the hem that adds a subtle touch of personality without being too loud. It's made from a cotton blend which makes it super comfortable and easy to care for. I love that it's versatile enough to pair with jeans for a casual look or with a blazer for a more put-together outfit."
25,average,hat,My favorite hat is a cozy knit beanie in a light gray color. It's perfect for keeping me warm during the colder months and goes well with most of my outfits.
25,average,hat,"My favorite hat is a cozy, chunky knit beanie that I found online last winter. It's a beautiful shade of dusty rose, which goes well with most of my wardrobe. It's super warm and has a ribbed texture, with a cute little faux fur pom-pom on top. I love how it keeps me warm while adding a stylish touch to my outfits on chilly days."


### Prompts
We can examine the default prompts for each question type by selecting them in the results. The following command will print both the system and user prompts for each question in the results. For convenience, we filter results to a single agent and model:

In [34]:
(results
.filter("agent_name == 'Agent_0' and model.model == 'gpt-4-1106-preview'")
.select("item", "prompt.*")
.print()
)

scenario.item,prompt.favorite_user_prompt,prompt.favorite_system_prompt
t-shirt,"{'text': 'You are being asked the following question: Describe your favorite t-shirt.\nReturn a valid JSON formatted like this:\n{""answer"": """"}', 'class_name': 'FreeText'}","{'text': ""You are answering questions as if you were a human. Do not break character. You are an agent with the following persona:\n{'persona': 'You are a 25-year-old short woman who prefers to shop online.', 'age': 25, 'height': 'short'}"", 'class_name': 'AgentInstruction'}"
hat,"{'text': 'You are being asked the following question: Describe your favorite hat.\nReturn a valid JSON formatted like this:\n{""answer"": """"}', 'class_name': 'FreeText'}","{'text': ""You are answering questions as if you were a human. Do not break character. You are an agent with the following persona:\n{'persona': 'You are a 25-year-old short woman who prefers to shop online.', 'age': 25, 'height': 'short'}"", 'class_name': 'AgentInstruction'}"
sweater,"{'text': 'You are being asked the following question: Describe your favorite sweater.\nReturn a valid JSON formatted like this:\n{""answer"": """"}', 'class_name': 'FreeText'}","{'text': ""You are answering questions as if you were a human. Do not break character. You are an agent with the following persona:\n{'persona': 'You are a 25-year-old short woman who prefers to shop online.', 'age': 25, 'height': 'short'}"", 'class_name': 'AgentInstruction'}"


### Analysis & visualization
The EDSL libary has a variety of methods for analyzing and visualizing results.

#### Dataframes
Results objects can be converted to pandas dataframes with the `to_pandas` method:

In [35]:
results.to_pandas()

,agent.age,agent.agent_name,agent.height,agent.persona,answer.favorite,iteration.iteration,model.frequency_penalty,model.logprobs,model.max_tokens,model.model,model.presence_penalty,model.temperature,model.top_logprobs,model.top_p,prompt.favorite_system_prompt,prompt.favorite_user_prompt,raw_model_response.favorite_raw_model_response,scenario.item
0,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,My favorite t-shirt is a cozy oversized black ...,0,0,False,1000,gpt-3.5-turbo,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3Bs1JWCMQLFU6A1XgNuLsyusNQe...,t-shirt
1,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,"My favorite t-shirt is a soft, vintage-style b...",0,0,False,1000,gpt-4-1106-preview,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BXyr3xiD9E6arLqUQuVltCwKuI...,t-shirt
2,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,My favorite hat is a cozy beanie with a cute p...,0,0,False,1000,gpt-3.5-turbo,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BsSRX73VaQCs9gRMxV2CwN78x0...,hat
3,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,"My favorite hat is a cozy, knit beanie with a ...",0,0,False,1000,gpt-4-1106-preview,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BX307TcUebeAMTJj5TI4HUFQ0A...,hat
4,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,My favorite sweater is a cozy oversized knit i...,0,0,False,1000,gpt-3.5-turbo,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BsBeiGuQwHl4l748E26Lhcg357...,sweater
5,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,"My favorite sweater is a cozy, oversized knit ...",0,0,False,1000,gpt-4-1106-preview,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BXejSQz5IchE8TNClEsrnyvLX6...,sweater
6,25,Agent_1,average,You are a 25-year-old average woman who prefer...,"My favorite t-shirt is a soft, oversized white...",0,0,False,1000,gpt-3.5-turbo,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BsIxt120pqBR1Pxog5YibP4KVO...,t-shirt
7,25,Agent_1,average,You are a 25-year-old average woman who prefer...,"My favorite t-shirt is a soft, heather gray nu...",0,0,False,1000,gpt-4-1106-preview,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BXIIRHIjbmzccPmxB8X2y8SlS8...,t-shirt
8,25,Agent_1,average,You are a 25-year-old average woman who prefer...,My favorite hat is a cozy knit beanie in a lig...,0,0,False,1000,gpt-3.5-turbo,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BsQ0bjZMsMD3tamdKJ1b4d0SKx...,hat
9,25,Agent_1,average,You are a 25-year-old average woman who prefer...,"My favorite hat is a cozy, chunky knit beanie ...",0,0,False,1000,gpt-4-1106-preview,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BXRxR41gYWGOBtLr1sllvKZXAd...,hat


#### SQL 
Results objects support SQL queries with the `sql` method.:

In [36]:
results.sql("select * from self", shape="wide")

,agent.age,agent.agent_name,agent.height,agent.persona,answer.favorite,iteration.iteration,model.frequency_penalty,model.logprobs,model.max_tokens,model.model,model.presence_penalty,model.temperature,model.top_logprobs,model.top_p,prompt.favorite_system_prompt,prompt.favorite_user_prompt,raw_model_response.favorite_raw_model_response,scenario.item
0,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,My favorite t-shirt is a cozy oversized black ...,0,0,0,1000,gpt-3.5-turbo,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3Bs1JWCMQLFU6A1XgNuLsyusNQe...,t-shirt
1,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,"My favorite t-shirt is a soft, vintage-style b...",0,0,0,1000,gpt-4-1106-preview,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BXyr3xiD9E6arLqUQuVltCwKuI...,t-shirt
2,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,My favorite hat is a cozy beanie with a cute p...,0,0,0,1000,gpt-3.5-turbo,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BsSRX73VaQCs9gRMxV2CwN78x0...,hat
3,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,"My favorite hat is a cozy, knit beanie with a ...",0,0,0,1000,gpt-4-1106-preview,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BX307TcUebeAMTJj5TI4HUFQ0A...,hat
4,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,My favorite sweater is a cozy oversized knit i...,0,0,0,1000,gpt-3.5-turbo,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BsBeiGuQwHl4l748E26Lhcg357...,sweater
5,25,Agent_0,short,You are a 25-year-old short woman who prefers ...,"My favorite sweater is a cozy, oversized knit ...",0,0,0,1000,gpt-4-1106-preview,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BXejSQz5IchE8TNClEsrnyvLX6...,sweater
6,25,Agent_1,average,You are a 25-year-old average woman who prefer...,"My favorite t-shirt is a soft, oversized white...",0,0,0,1000,gpt-3.5-turbo,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BsIxt120pqBR1Pxog5YibP4KVO...,t-shirt
7,25,Agent_1,average,You are a 25-year-old average woman who prefer...,"My favorite t-shirt is a soft, heather gray nu...",0,0,0,1000,gpt-4-1106-preview,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BXIIRHIjbmzccPmxB8X2y8SlS8...,t-shirt
8,25,Agent_1,average,You are a 25-year-old average woman who prefer...,My favorite hat is a cozy knit beanie in a lig...,0,0,0,1000,gpt-3.5-turbo,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BsQ0bjZMsMD3tamdKJ1b4d0SKx...,hat
9,25,Agent_1,average,You are a 25-year-old average woman who prefer...,"My favorite hat is a cozy, chunky knit beanie ...",0,0,0,1000,gpt-4-1106-preview,0,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9E3BXRxR41gYWGOBtLr1sllvKZXAd...,hat


We can show the schema of the results with the `show_schema` method, which requires a `shape` argument (`shape="long"` or `shape="wide"`).

Note that using `shape="wide"` in the `sql` method above displayed all columns horizontally, whereas `shape="long"` displays them vertically with key-value pairs of column names and values:

In [37]:
results.show_schema(shape="long")

Type: table, Name: self, SQL: CREATE TABLE self (
                id INTEGER,
                data_type TEXT,
                key TEXT, 
                value TEXT
            )



In [38]:
results.sql("select * from self", shape="long")

,id,data_type,key,value
0,0,agent,persona,You are a 25-year-old short woman who prefers ...
1,0,agent,age,25
2,0,agent,height,short
3,0,agent,agent_name,Agent_0
4,0,scenario,item,t-shirt
...,...,...,...,...
1021,53,prompt,favorite_user_prompt,{'text': 'You are being asked the following qu...
1022,53,prompt,favorite_system_prompt,"{'text': ""You are answering questions as if yo..."
1023,53,raw_model_response,favorite_raw_model_response,{'id': 'chatcmpl-9E3BXvCcof4uKFCMSVQFZH0eMNtX0...
1024,53,iteration,iteration,0


#### Visualizations
Built-in visualization methods include: 
`.word_cloud_plot()` 
`.bar_chart()` 
`.faceted_bar_chart()` 
`.histogram_plot()` 

and an interactive html method: 
`.select(...).print(html=True, pretty_labels = {...}, interactive = True)` 

To visualize a subset of results, we can first apply the `filter` method.

In [39]:
results.word_cloud_plot("favorite")

In [40]:
results_shopping.bar_chart("q_mc", title = q_mc.question_text)

In [41]:
q_mc = QuestionMultipleChoice(
    question_name = "q_mc",
    question_text = "How often do you shop for {{ item }}?",
    question_options = [
        "Rarely or never",
        "Annually",
        "Seasonally",
        "Monthly",
        "Daily"
    ]
)
scenarios = [Scenario({"item": i}) for i in ["books", "electronics", "vacation homes"]]
results = q_mc.by(scenarios).run()

results.faceted_bar_chart("q_mc", "item", title = "Shopping frequency")

In [42]:
q_ls = QuestionLinearScale(
    question_name = "q_ls",
    question_text = "On a scale of 0-10, how much do you typically enjoy shopping for {{ item }}? (0 = Not at all, 10 = Very much)",
    question_options = [0,1,2,3,4,5,6,7,8,9,10]
)
scenarios = [Scenario({"item": i}) for i in ["books", "electronics", "vacation homes"]]
agents = [Agent(traits = {"persona": p}) for p in ["You are a 45-year-old woman who prefers to shop online.", "You are a 25-year-old man who prefers to shop in person."]]
results = q_ls.by(scenarios).by(agents).run()

results.filter("item == 'books'").histogram_plot("q_ls", title = "Shopping enjoyment")

---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>